## MIPHA test on a real use case

The purpose of this notebook is to test the MIPHA framework with a real use case. A proper "example" package will be provided in a future version, once the framework has stabilized.

The data used in this example has been extracted from the [MIMIC-IV database](https://physionet.org/content/mimiciv/2.2/). The code used to extract the data will also be added to the example package.

In [2]:
import sys
from importlib import reload

import pandas as pd 

import tests.test_real_data.real_data_manager as data
import tests.test_real_data.simple_implementation as impl # necessary for pickle to work, cannot use from ... import *
from src.mipha.framework import MiphaPredictor

In [3]:
reload(sys.modules['src.mipha.framework'])
reload(sys.modules['tests.test_real_data.simple_implementation'])
reload(sys.modules['tests.test_real_data.real_data_manager'])

<module 'tests.test_real_data.real_data_manager' from 'C:\\Biologie_PhD\\mipha\\tests\\test_real_data\\real_data_manager.py'>

### Framework implementation

We test a simple implementation of the framework applied to the prediction of stage 5 CKD (using a year of history for a prediction up to 15 months in advance).
The data sources used in this example are:
- The evolution of creatinine over time.
- The age and gender of the patient.

The framework is implemented as such:
- Feature extraction for the first data source is performed using the `tsfel` package.
- Aggregation is a simple concatenation of the extracted features.
- The machine learning model is a simple CNN.

In [ ]:
data_sources_train, labels_train, data_sources_test, labels_test = data.load_stage_5_ckd(random_state=25)

In [ ]:
mipha = MiphaPredictor(
    feature_extractors=[
        impl.BiologyFeatureExtractor(component_name="BiologyFeatureExtractor", managed_data_types=["Creatinine"]),
        impl.DemographicsFeatureExtractor(component_name="BiologyFeatureExtractor",
                                          managed_data_types=["Demographics"]),
    ],
    aggregator=impl.SimpleAggregator(),
    model=impl.SimpleCnnModel(rows=1, columns=142, output_dim=1, n_filters=3),
    # input dimensions are picked for the aggregator, output is binary
    evaluator=impl.SimpleEvaluator(),
)

In [ ]:
mipha.fit(data_sources_train, labels_train, epochs=3)

In [ ]:
mipha.evaluate(data_sources=data_sources_test, test_labels=pd.DataFrame(labels_test), threshold=0.5)

In [ ]:
from datetime import datetime

now = datetime.now()
formatted_time = now.strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"out/mipha_real_data_{formatted_time}"
mipha.save(file_path)

In [ ]:
# Reusing a feature extraction

# data.save_pickle(mipha.last_computed_features, "out/mipha_computed_features.pkl")

mipha_loaded = mipha.load("out/mipha_real_data.zip")
precomputed_features = data.load_pickle("out/mipha_computed_features.pkl")

In [ ]:
mipha.evaluate(data_sources_test, labels_test, precomputed_features=precomputed_features)